In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=2897c1d5b396774f86db2c30d8e532b332336638531bca68971c4639cc878a2c
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assesment.com").getOrCreate()

In [ ]:
df = spark.read.csv("titanic.csv", header=True)

In [ ]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
#A.How many people survive and how many people dead
# count the number of survivors and deaths
#A -> Spark
survived_count = df.filter(df.Survived == 1).count()
death_count = df.filter(df.Survived == 0).count()

In [ ]:
print("Survivors:", survived_count)

Survivors: 342


In [ ]:
print("Deaths:", death_count)

Deaths: 549


In [ ]:
df.createOrReplaceTempView("titanic_table")

In [ ]:
#A -> SQL
# count the number of survivors
survivors = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 1").collect()[0][0]
print(f"Number of survivors: {survivors}")

# count the number of deaths
deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0").collect()[0][0]
print(f"Number of deaths: {deaths}")


Number of survivors: 342
Number of deaths: 549


In [ ]:
#B.How many people survive how does not have sibling
#B -> Spark
from pyspark.sql.functions import col
# count the number of survivors without siblings
survived_without_siblings_count = df.filter((df.Survived == 1) & (col("SibSp") == 0)).count()
print("Survivors without siblings:", survived_without_siblings_count)


Survivors without siblings: 210


In [ ]:
#B -> SQL
survivors_without_siblings_count = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 1 AND SibSp = 0").collect()[0][0]
print(f"Number of survivors without siblings: {survivors_without_siblings_count}")

Number of survivors without siblings: 210


In [ ]:
#C.How many people died Who does not have sibling
# count the number of death without siblings
#C ->Spark
death_without_siblings_count = df.filter((df.Survived == 0) & (col("SibSp") == 0)).count()
print("Died without siblings:", death_without_siblings_count)

Died without siblings: 398


In [ ]:
#C ->SQL
death_without_siblings = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND SibSp = 0").collect()[0][0]
print(f"Number of survivors without siblings: {death_without_siblings}")

Number of survivors without siblings: 398


In [ ]:
#D.Remove the NAN data from cabin and display updated list
#D -Spark
#Drop the row if that row contains null value in "Cabin" Column
df_without_nan_cabin = df.na.drop(subset=["Cabin"])
df_without_nan_cabin.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|  54|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|   4|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|  58|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|  34|  

In [ ]:
#D -Sql
# remove rows with null values in the 'cabin' column
df_filtered = spark.sql("SELECT * FROM titanic_table WHERE cabin IS NOT NULL")

# display the updated list
df_filtered.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|  54|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|   4|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|  58|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|  34|  

In [ ]:
#E.How many male who died and how many female died?
#E -> Spark
# Group the data by Sex and Survived columns and count the number of deaths
from pyspark.sql.functions import count
male_deaths = df.filter((df['Survived'] == 0) & (df['Sex'] == 'male')).agg(count('*')).collect()[0][0]
female_deaths = df.filter((df['Survived'] == 0) & (df['Sex'] == 'female')).agg(count('*')).collect()[0][0]
print(f"Number of male deaths: {male_deaths}")
print(f"Number of female deaths: {female_deaths}")


Number of male deaths: 468
Number of female deaths: 81


In [ ]:
#E -> SQL
# count the number of males who died
male_deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND Sex = 'male'").collect()[0][0]
print(f"Number of male deaths: {male_deaths}")

Number of male deaths: 468


In [ ]:
# count the number of females who died
female_deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND Sex = 'female'").collect()[0][0]
print(f"Number of female deaths: {female_deaths}")

Number of female deaths: 81
